In [3]:
from vision_transformers.detection.rtdetr.src.zoo.rtdetr_model import RTDETR

In [5]:
model = RTDETR(None, None, None)

TypeError: PResNet.__init__() missing 1 required positional argument: 'depth'